# **Instituto Tecnológico Vale**

## **Introdução a Visão Computacional**

### **Aula 05 - Reconhecimento de imagem**

## **Exercício 1**

### Atualize o exemplo prático de reconhcimento de logos para reconhecer o logo da Vale.

- Crie as novas pastas nos locais corretos e insira as novas imagens do logo Vale.

- Execute o treinamento conforme a prática

- Na fase da previsão de teste, se a previsão for do logo Vale escreva o nome da em verde, caso contrário escreva o nome da marca em amarelo.


### **Enviar até o dia 03/09 por email**

**- Código está disponível abaixo abaixo. Edite onde for indicado.**

In [ ]:
# Imports

import argparse
import imutils
import cv2
from sklearn.neighbors import KNeighborsClassifier
from skimage import exposure
from skimage import feature
from imutils import paths
import numpy as np

from google.colab.patches import cv2_imshow

In [ ]:
# Baixar os arquivos necessários para a aula
!rm -rf intro_cv_proficam
!git clone https://github.com/h3ct0r/intro_cv_proficam
!cp -r intro_cv_proficam/img img

In [ ]:
### Colocar o seu codigo aqui!!!

# Crie as variaveis para os dados e labels

In [ ]:
# Loop pelas imagens no dataset de treino

for imagePath in paths.list_images("img/pratica_logos/logos"):
	make = imagePath.split("/")[-2]
	image = cv2.imread(imagePath)
	gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
	edged = imutils.auto_canny(gray)

	# Encontra contornos no mapa de borda, mantendo apenas o maior que se supõe ser o logotipo do carro
	(cnts, _) = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
	c = max(cnts, key=cv2.contourArea)

	# Extrai o logotipo do carro e redimensiona 
	(x, y, w, h) = cv2.boundingRect(c)
	logo = gray[y:y + h, x:x + w]
	logo = cv2.resize(logo, (200, 100))

	### Colocar o seu codigo aqui!!!
  # execute o algoritmo hog para extração das features

	# Atualiza dados e labels
	data.append(H)
	labels.append(make)

In [ ]:
### Colocar o seu codigo aqui!!!

# Crie o modelo do KNeighborsClassifier e treine

In [ ]:
# Loop no dataset de teste
print ("Avaliando...")
for (i, imagePath) in enumerate(paths.list_images("img/pratica_logos/test_images")):
	image = cv2.imread(imagePath)
	gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
	logo = cv2.resize(gray, (200, 100))

	# Extrai o Histograma de Gradientes Orientados da imagem de teste e prevê a marca do carro
	H = feature.hog(logo, orientations=9, pixels_per_cell=(10, 10), cells_per_block=(2, 2), transform_sqrt=True)
	pred = model.predict(H.reshape(1, -1))[0]

	### Colocar o seu codigo aqui!!!
  faça o print correto das previsões considerando a cor do texto sugerida para o exercício

## **Exercício 2**

### Construa sua rede convolucional para classificação das imagens de ants e bees

- Não use o mesmo modelo criado em aula.

- Teste os hiperparâmetros para melhorar seu resultado.

- Descreva a estrutura final da sua rede.

- Utilize as células do colab para descrever como foi o processo de treino e quais mudanças tiveram mais efeito nos resultados.

- Comente sobre as dificuldades encontradas e o que pode ser feito para melhorar os resultados.

**- Código está disponível abaixo abaixo. Edite onde for indicado.**

In [ ]:
# baixar e descompactar os dados do exemplo
# Será criado a pasta hymenoptera_data no ambiente do colab

!wget -O hymenoptera_data.zip https://www.dropbox.com/s/154i61alh62e7jl/hymenoptera_data.zip?dl=0;

!unzip hymenoptera_data.zip;

In [ ]:
#imports necessários

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

In [ ]:
### Atenção célula editável!!!

# Data augmentation and normalization for training
# Just normalization for validation

# Dica: veja sugestões de mais transformações para data augmentation em https://pytorch.org/vision/stable/transforms.html

data_transforms = {
    'train': transforms.Compose([
                                 # Coloque o seu data augmentation aqui!!!
                                 transforms.ToTensor(),
                                 transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [ ]:
### Atenção célula editável!!!


# Crie os datasets e dataloaders específicos

data_dir = 'hymenoptera_data'



dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(f'Dataset sizes: {dataset_sizes}')
print(f'class names: {class_names}')
print(f'cuda: {device}')

In [ ]:
# Funções auxiliares para a prática

# Função auxiliar para impimir um batch de imagens

def imshow(inp, title=None):
    """Imshow for Tensor."""

    #plt.figure(figsize=(10, 8))

    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated

In [ ]:
# Get a batch of training data
inputs, classes = next(iter(dataloaders['train']))

# Make a grid from batch
out = torchvision.utils.make_grid(inputs)

imshow(out, title=[class_names[x] for x in classes])

In [ ]:
### Atenção célula editável!!!

# função de treino

def train_model(model, criterion, optimizer, num_epochs=15):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    ###Colocar o seu codigo aqui!!!
                    # Passe a imagem pela rede e calcule o erro

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        ###Colocar o seu codigo aqui!!!
                        # atualize os pesos

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

In [ ]:
### Atenção célula editável!!!

# Crie a sua CNN

class classificador(nn.Module):

  def __init__(self):
    super().__init__()

    # Coloque seu código aqui


  # Fluxo da passagem da imagem na rede 
  def forward(self, X):

    # Coloque seu código aqui

In [ ]:
### Atenção célula editável!!!

# Crie o objeto do classificador


In [ ]:
###Colocar o seu codigo aqui!!!

# Defina sua loss function e o otimizador

In [ ]:
###Colocar o seu codigo aqui!!!

# Coloque a rede na gpu

In [ ]:
### Atenção célula editável!!!

# Define a quantidade de epocas e treine o modelo

trained_model = train_model(net, criterion, optimizer, num_epochs=)

In [ ]:
# Visualizar imagens do conjunto de teste classificadas
visualize_model(trained_model)

### Descreva seus resultados e faça os comentários abaixo: